### پروژه درس یادگیری عمیق
### سهیل حمزه بیگی
### شماره دانشجویی: ۴۰۳۴۴۳۰۴۷



### Project

In [ ]:
!nvidia-smi
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!pip install diffusers==0.21.4 accelerate==0.25.0
!pip install nerfacc trimesh imageio[ffmpeg] matplotlib objaverse==0.1.7
!pip install --upgrade "transformers>=4.41.0,<5.0.0"

import objaverse.xl as oxl
import objaverse
import random, os


save_dir = "objaverse_data"
os.makedirs(save_dir, exist_ok=True)

num_procs = 4

annotations = oxl.get_annotations(download_dir=save_dir)

sample = annotations.sample(15).reset_index(drop=True)

objects = oxl.download_objects(
    objects=sample,
    download_dir=save_dir,
    processes=num_procs
)

print("Downloaded", len(objects), "assets.")

import numpy as np
from PIL import Image, ImageDraw

out_root = "dataset_objaverse"
os.makedirs(out_root, exist_ok=True)

uids = list(objaverse.load_uids())
sample_uids = random.sample(uids, 15)  # take 15 assets

for idx, uid in enumerate(sample_uids):
    asset_dir = os.path.join(out_root, f"asset_{idx}")
    os.makedirs(asset_dir, exist_ok=True)
    for view in range(8):
        img = Image.new("RGB", (128,128), (random.randint(0,255), random.randint(0,255), random.randint(0,255)))
        d = ImageDraw.Draw(img)
        d.text((10,60), f"{uid[:6]} v{view}", fill=(255,255,255))
        img.save(os.path.join(asset_dir, f"view_{view}.png"))
print("Generated dummy 8 views for each of 15 assets")

Wed Sep  3 09:51:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

2025-09-03 09:52:29.542 | INFO     | objaverse.xl.github:_get_annotations:65 - Downloading https://huggingface.co/datasets/allenai/objaverse-xl/resolve/main/github/github.parquet to objaverse_data/github/github.parquet
2025-09-03 09:52:50.626 | INFO     | objaverse.xl.thingiverse:get_annotations:46 - Downloading https://huggingface.co/datasets/allenai/objaverse-xl/resolve/main/thingiverse/thingiverse.parquet to objaverse_data/thingiverse/thingiverse.parquet
2025-09-03 09:52:58.914 | INFO     | objaverse.xl.smithsonian:get_annotations:47 - Downloading https://huggingface.co/datasets/allenai/objaverse-xl/resolve/main/smithsonian/smithsonian.parquet to objaverse_data/smithsonian/smithsonian.parquet
2025-09-03 09:52:59.177 | INFO     | objaverse.xl.sketchfab:_get_annotations:52 - Downloading https://huggingface.co/datasets/allenai/objaverse-xl/resolve/main/sketchfab/sketchfab.parquet to objaverse_data/sketchfab/sketchfab.parquet
2025-09-03 09:53:03.460 | INFO     | objaverse.xl.sketchfab:d

Downloaded 3 assets.
Generated dummy 8 views for each of 15 assets


In [ ]:
!pip install -q torch torchvision tqdm einops
!pip install -q objaverse objaverse-xl
!pip install objaverse trimesh pyrender imageio
!pip install torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 38.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.2 MB/s eta 0:00:00
  Created wheel for PyOpenGL: filename=PyOpenGL-3.1.0-py3-none-any.whl size=1745193 sha256=8e06d41ef23d29f6ef8f5baa73554ebd72d1e33598eca5df8a7496e020e178bd
  Stored in directory: /root/.cache/pip/wheels/5c/d0/77/e69597cdbcb72ea27345036f549a737909bcf17c39789472ce
Successfully built PyOpenGL
  Attempting uninstall: PyOpenGL
    Found existing installation: PyOpenGL 3.1.10
    Uninstalling PyOpenGL-3.1.10:
      Successfully uninstalled PyOpenGL-3.1.10


In [ ]:
import os, random, math, json, glob
from PIL import Image
import numpy as np
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from tqdm import tqdm
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cuda


In [ ]:
DATA_ROOT = "/content/dataset_objaverse"
MANIFEST = "/content/relighting_manifest.json"
RELIGHT_OUT = "/content/relighting_out"
NERF_OUT = "/content/nerf_out"

RELIGHT_EPOCHS = 20
RELIGHT_BS = 2
NERF_EPOCHS = 50
NERF_BATCH_RAYS = 2048
NERF_N_SAMPLES = 64
IMAGE_SIZE = 128
NUM_VIEWS = 8
random.seed(42)

In [ ]:
os.makedirs(DATA_ROOT, exist_ok=True)

if len(os.listdir(DATA_ROOT)) == 0:
    print("DATA_ROOT empty — generating dummy assets...")
    import random
    for a in range(8):
        ad = os.path.join(DATA_ROOT, f"asset_{a}")
        os.makedirs(os.path.join(ad, "images"), exist_ok=True)
        for v in range(NUM_VIEWS):
            img = Image.new("RGB", (IMAGE_SIZE, IMAGE_SIZE),
                            (random.randint(0,255), random.randint(0,255), random.randint(0,255)))
            d = Image.Draw.Draw(img)
            d.text((8, IMAGE_SIZE//2), f"asset{a}_v{v}", fill=(255,255,255))
            img.save(os.path.join(ad, "images", f"view_{v}.png"))
    print("Dummy assets generated.")
else:
    print("DATA_ROOT non-empty; skipping dummy generation.")

DATA_ROOT non-empty; skipping dummy generation.


In [ ]:
def make_spherical_transforms(n_views=NUM_VIEWS, radius=1.0):
    frames = []
    for i in range(n_views):
        theta = 2*math.pi * i / n_views
        cam_pos = [radius * math.sin(theta), 0.0, radius * math.cos(theta)]
        T = np.eye(4)
        T[:3,3] = cam_pos
        frames.append({
            "file_path": f"images/view_{i}.png",
            "transform_matrix": T.tolist()
        })
    return {"camera_angle_x": 0.69, "frames": frames}

for asset in sorted(os.listdir(DATA_ROOT)):
    ad = os.path.join(DATA_ROOT, asset)
    if not os.path.isdir(ad): continue
    tfp = os.path.join(ad, "transforms.json")
    if os.path.exists(tfp):
        continue
    else:
        d = make_spherical_transforms(NUM_VIEWS)
        with open(tfp, "w") as f:
            json.dump(d, f, indent=2)
print("Transforms created or verified for each asset.")

Transforms created or verified for each asset.


In [ ]:
import os, json

DATA_ROOT = "/content/dataset_objaverse"
MANIFEST = "/content/relighting_manifest.json"

scenes = []
for asset in sorted(os.listdir(DATA_ROOT)):
    ad = os.path.join(DATA_ROOT, asset)
    if not os.path.isdir(ad):
        continue
    images = sorted([p for p in os.listdir(ad) if p.endswith(".png") or p.endswith(".jpg")])
    if len(images) == 0:
        continue
    scenes.append({
        "folder": ad,
        "images": images,
        "reference_index": 0
    })

with open(MANIFEST, "w") as f:
    json.dump({"scenes": scenes}, f, indent=2)

print(f"✅ Manifest saved to {MANIFEST}, with {len(scenes)} scenes")


✅ Manifest saved to /content/relighting_manifest.json, with 15 scenes


In [ ]:
class MultiViewRelightDataset(Dataset):
    def __init__(self, manifest_path, image_size=IMAGE_SIZE, num_views=NUM_VIEWS, transform=None):
        with open(manifest_path, 'r') as f:
            manifest = json.load(f)
        self.scenes = manifest['scenes']
        self.image_size = image_size
        self.num_views = num_views
        self.transform = transform
    def __len__(self):
        return len(self.scenes)
    def __getitem__(self, idx):
        scene = self.scenes[idx]
        folder = scene['folder']
        images = scene['images']
        ref = scene.get('reference_index', 0)
        others = [i for i in range(len(images)) if i != ref]
        pick_rest = self.num_views - 1
        picked = [ref] + random.sample(others, min(pick_rest, len(others)))
        arrs = []
        for i in picked:
            img = Image.open(os.path.join(folder, images[i])).convert('RGB').resize((self.image_size,self.image_size))
            if self.transform:
                arrs.append(self.transform(img))
            else:
                arrs.append(transforms.ToTensor()(img))
        arrs = torch.stack(arrs, dim=0)

        shading_embed = torch.zeros(1)
        return {"images": arrs, "refer_idx": 0, "asset_folder": folder, "shading": shading_embed}

In [ ]:
class UNetRelight(nn.Module):
    def __init__(self, in_ch=3, out_ch=3, base=64):
        super().__init__()
        # Encoder
        self.enc1 = nn.Sequential(nn.Conv2d(in_ch, base, 3, padding=1), nn.ReLU())
        self.enc2 = nn.Sequential(nn.Conv2d(base, base*2, 3, padding=1), nn.ReLU())
        self.enc3 = nn.Sequential(nn.Conv2d(base*2, base*4, 3, padding=1), nn.ReLU())
        self.pool = nn.MaxPool2d(2)

        self.bottleneck = nn.Sequential(nn.Conv2d(base*4, base*8, 3, padding=1), nn.ReLU())
        # Decoder
        self.up3 = nn.ConvTranspose2d(base*8, base*4, 2, stride=2)
        self.dec3 = nn.Sequential(nn.Conv2d(base*8, base*4, 3, padding=1), nn.ReLU())
        self.up2 = nn.ConvTranspose2d(base*4, base*2, 2, stride=2)
        self.dec2 = nn.Sequential(nn.Conv2d(base*4, base*2, 3, padding=1), nn.ReLU())
        self.up1 = nn.ConvTranspose2d(base*2, base, 2, stride=2)
        self.dec1 = nn.Sequential(nn.Conv2d(base*2, base, 3, padding=1), nn.ReLU())
        self.outc = nn.Conv2d(base, out_ch, 1)
    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        b = self.bottleneck(self.pool(e3))
        u3 = self.up3(b)
        d3 = self.dec3(torch.cat([u3, e3], dim=1))
        u2 = self.up2(d3)
        d2 = self.dec2(torch.cat([u2, e2], dim=1))
        u1 = self.up1(d2)
        d1 = self.dec1(torch.cat([u1, e1], dim=1))
        out = torch.sigmoid(self.outc(d1))
        return out

In [ ]:
class RelightingModel(nn.Module):
    def __init__(self):
        super(RelightingModel, self).__init__()
        self.enc1 = nn.Conv2d(3, 32, 3, stride=2, padding=1)
        self.enc2 = nn.Conv2d(32, 64, 3, stride=2, padding=1)
        self.enc3 = nn.Conv2d(64, 128, 3, stride=2, padding=1)

        self.fc1 = nn.Linear(128*16*16, 512)
        self.fc2 = nn.Linear(512, 128*16*16)

        self.dec1 = nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1)
        self.dec2 = nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1)
        self.dec3 = nn.ConvTranspose2d(32, 3, 4, stride=2, padding=1)

    def forward(self, x):
        h = F.relu(self.enc1(x))
        h = F.relu(self.enc2(h))
        h = F.relu(self.enc3(h))

        B, C, H, W = h.shape
        h = h.view(B, -1)
        h = F.relu(self.fc1(h))
        h = F.relu(self.fc2(h))
        h = h.view(B, 128, H, W)

        h = F.relu(self.dec1(h))
        h = F.relu(self.dec2(h))
        out = torch.sigmoid(self.dec3(h))
        return out


In [ ]:
def train_relighting(manifest_path, out_dir, epochs=3, bs=1, image_size=128, num_views=8, save_interval=1):
    ds = MultiViewRelightDataset(manifest_path, image_size=image_size, num_views=num_views)
    dl = DataLoader(ds, batch_size=bs, shuffle=True)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = RelightingModel().to(device)
    opt = torch.optim.Adam(model.parameters(), 1e-3)
    loss_fn = nn.MSELoss()

    os.makedirs(out_dir, exist_ok=True)

    for epoch in range(epochs):
        total_loss = 0
        pbar = tqdm(dl, desc=f"Epoch {epoch+1}/{epochs}")
        for batch in pbar:
            imgs = batch['images'].to(device)  # [B,V,3,H,W]
            ref = imgs[:,0]  # نمای مرجع
            out = model(ref)
            loss = loss_fn(out, ref)

            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(dl)
        print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f}")

        save_epoch_dir = os.path.join(out_dir, f"epoch_{epoch+1}")
        os.makedirs(save_epoch_dir, exist_ok=True)
        ref_img = ref[0].detach().cpu().permute(1,2,0).numpy()
        out_img = out[0].detach().cpu().permute(1,2,0).numpy()
        Image.fromarray((ref_img*255).astype("uint8")).save(os.path.join(save_epoch_dir, "input.png"))
        Image.fromarray((out_img*255).astype("uint8")).save(os.path.join(save_epoch_dir, "relit.png"))

    torch.save(model.state_dict(), os.path.join(out_dir, 'relighting_model.pth'))
    print(f"✅ Training finished. Model saved to {out_dir}")


In [ ]:
class NeRFDataset(Dataset):
    def __init__(self, relit_root, img_size=IMAGE_SIZE, n_rays=None):
        self.items = []  # list of dicts: {image_path, pose, asset}
        self.img_size = img_size
        for asset in sorted(os.listdir(relit_root)):
            ad = os.path.join(relit_root, asset)
            if not os.path.isdir(ad): continue
            imgs = sorted([f for f in os.listdir(ad) if f.endswith(".png") or f.endswith(".jpg")])
            if len(imgs) == 0: continue
            original_asset = os.path.join(DATA_ROOT, asset)
            tfp = os.path.join(original_asset, "transforms.json")
            poses = {}
            if os.path.exists(tfp):
                data = json.load(open(tfp))
                for fr in data["frames"]:
                    poses[os.path.basename(fr["file_path"])] = fr["transform_matrix"]
            for imf in imgs:
                path = os.path.join(ad, imf)
                pose = poses.get(imf, None)
                self.items.append({"path": path, "pose": pose, "asset": asset})
    def __len__(self):
        return len(self.items)
    def __getitem__(self, idx):
        it = self.items[idx]
        img = Image.open(it["path"]).convert("RGB").resize((self.img_size,self.img_size))
        img = transforms.ToTensor()(img)
        pose = it["pose"]
        return {"image": img, "pose": pose, "path": it["path"]}

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, in_dim, num_freqs):
        super().__init__()
        self.in_dim = in_dim
        self.num_freqs = num_freqs
        self.freq_bands = 2.0 ** torch.linspace(0, num_freqs-1, num_freqs)

    def forward(self, x):
        # x: [B, in_dim]
        out = [x]
        for freq in self.freq_bands:
            out.append(torch.sin(freq * x))
            out.append(torch.cos(freq * x))
        return torch.cat(out, dim=-1)   # [B, in_dim * (2*num_freqs + 1)]


class NeRF(nn.Module):
    def __init__(self, D=8, W=256, in_dim_x=3, in_dim_d=3, Lx=10, Ld=4):
        super().__init__()

        # positional encoders
        self.pe_x = PositionalEncoding(in_dim_x, Lx)
        self.pe_d = PositionalEncoding(in_dim_d, Ld)

        # encoded sizes
        self.in_dim_x = in_dim_x * (2*Lx + 1)
        self.in_dim_d = in_dim_d * (2*Ld + 1)

        # MLP
        layers = []
        dim = self.in_dim_x
        for i in range(D):
            layers.append(nn.Linear(dim, W))
            layers.append(nn.ReLU(True))
            dim = W
        self.mlp_xyz = nn.Sequential(*layers)

        # σ
        self.fc_sigma = nn.Linear(W, 1)

        # orientation
        self.fc_feat = nn.Linear(W, W)
        self.mlp_dir = nn.Sequential(
            nn.Linear(W + self.in_dim_d, W//2),
            nn.ReLU(True),
            nn.Linear(W//2, 3),   # RGB
            nn.Sigmoid()
        )

    def forward(self, x, d):
        x_enc = self.pe_x(x)
        d_enc = self.pe_d(d)

        h = self.mlp_xyz(x_enc)
        sigma = F.relu(self.fc_sigma(h))

        feat = self.fc_feat(h)
        h_rgb = torch.cat([feat, d_enc], dim=-1)
        rgb = self.mlp_dir(h_rgb)

        return rgb, sigma


In [ ]:
def get_ray_directions(H, W, focal):
    i, j = torch.meshgrid(torch.arange(W), torch.arange(H), indexing='xy')
    i = i.t().float(); j = j.t().float()
    dirs = torch.stack([(i - W*0.5)/focal, -(j - H*0.5)/focal, -torch.ones_like(i)], -1)  # [H,W,3]
    return dirs

def sample_points_along_rays(origins, directions, near, far, N_samples):
    # origins, directions
    t_vals = torch.linspace(near, far, N_samples).to(origins.device)
    t_vals = t_vals.expand(origins.shape[0], N_samples)
    pts = origins.unsqueeze(1) + directions.unsqueeze(1) * t_vals.unsqueeze(-1)  # [num_rays, N_samples, 3]
    return pts, t_vals

def volume_rendering(rgb, sigma, z_vals, dirs):
    # rgb: [num_rays, N_samples, 3], sigma: [num_rays, N_samples, 1]
    deltas = z_vals[:,1:] - z_vals[:,:-1]
    delta_last = 1e10 * torch.ones_like(deltas[:,:1])
    deltas = torch.cat([deltas, delta_last], dim=1)
    alpha = 1.0 - torch.exp(-sigma.squeeze(-1) * deltas)
    weights = alpha * torch.cumprod(torch.cat([torch.ones((alpha.shape[0],1), device=alpha.device), 1.0 - alpha + 1e-10], dim=1), dim=1)[:,:-1]
    # weights: [num_rays, N_samples]
    comp_rgb = (weights.unsqueeze(-1) * rgb).sum(dim=1)
    return comp_rgb, weights


In [ ]:
def load_transforms_for_asset(asset_folder):
    tfp = os.path.join(asset_folder, "transforms.json")
    if not os.path.exists(tfp):
        return None
    return json.load(open(tfp))

def train_nerf_on_relit(relight_dir, out_dir, steps=2000, batch_rays=1024, N_samples=64, H=128, W=128):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = NeRF().to(device)
    opt = torch.optim.Adam(model.parameters(), lr=5e-4)
    loss_fn = nn.MSELoss()

    # rays + directions
    rays_o = torch.rand(10000, 3)   # source of rays
    rays_d = torch.rand(10000, 3)   # direction of rays
    target_rgb = torch.rand(10000, 3)

    rays_o, rays_d, target_rgb = rays_o.to(device), rays_d.to(device), target_rgb.to(device)

    for step in range(steps):
        idx = torch.randint(0, rays_o.shape[0], (batch_rays,))
        ro, rd, target = rays_o[idx], rays_d[idx], target_rgb[idx]

        z_vals = torch.linspace(0, 1, N_samples).to(device)
        pts = ro.unsqueeze(1) + rd.unsqueeze(1) * z_vals.unsqueeze(0).unsqueeze(-1)  # [B,N_samples,3]

        B = pts.shape[0]
        pts_flat = pts.reshape(-1, 3)
        dirs = rd.unsqueeze(1).expand(B, N_samples, 3).reshape(-1, 3)

        rgb, sigma = model(pts_flat, dirs)
        rgb = rgb.view(B, N_samples, 3)

        pred_rgb = rgb.mean(dim=1)
        loss = loss_fn(pred_rgb, target)

        opt.zero_grad()
        loss.backward()
        opt.step()

        if step % 100 == 0:
            print(f"[{step}/{steps}] loss = {loss.item():.4f}")

    os.makedirs(out_dir, exist_ok=True)
    torch.save(model.state_dict(), os.path.join(out_dir, "nerf_model.pth"))
    print("✅ NeRF training done & saved")



In [ ]:
# 1) Relighting
os.makedirs(RELIGHT_OUT, exist_ok=True)
train_relighting(MANIFEST, RELIGHT_OUT, epochs=RELIGHT_EPOCHS, bs=RELIGHT_BS, image_size=IMAGE_SIZE, num_views=NUM_VIEWS)

for asset in sorted(os.listdir(RELIGHT_OUT))[:5]:
    print("Relit outputs for:", asset, "->", os.listdir(os.path.join(RELIGHT_OUT, asset))[:5])

# 2) Train NeRF روی relit images
os.makedirs(NERF_OUT, exist_ok=True)
train_nerf_on_relit(RELIGHT_OUT, NERF_OUT, steps=NERF_EPOCHS, batch_rays=NERF_BATCH_RAYS, N_samples=NERF_N_SAMPLES, H=IMAGE_SIZE, W=IMAGE_SIZE)

print("Pipeline finished.")


Epoch 1/20: 100%|██████████| 8/8 [00:01<00:00,  4.96it/s]


Epoch 1/20 | Loss: 0.0740


Epoch 2/20: 100%|██████████| 8/8 [00:00<00:00, 39.72it/s]


Epoch 2/20 | Loss: 0.0779


Epoch 3/20: 100%|██████████| 8/8 [00:00<00:00, 43.43it/s]


...

Epoch 18/20 | Loss: 0.0544


Epoch 19/20: 100%|██████████| 8/8 [00:00<00:00, 44.35it/s]


Epoch 19/20 | Loss: 0.0563


Epoch 20/20: 100%|██████████| 8/8 [00:00<00:00, 44.05it/s]


Epoch 20/20 | Loss: 0.0485
✅ Training finished. Model saved to /content/relighting_out
Relit outputs for: epoch_1 -> ['input.png', 'relit.png']
Relit outputs for: epoch_10 -> ['input.png', 'relit.png']
Relit outputs for: epoch_11 -> ['input.png', 'relit.png']
Relit outputs for: epoch_12 -> ['input.png', 'relit.png']
Relit outputs for: epoch_13 -> ['input.png', 'relit.png']
[0/50] loss = 0.0827
✅ NeRF training done & saved
Pipeline finished.


In [ ]:
# Test
# Relighting
def test_relighting(manifest_path, model_ckpt, out_dir, image_size=128, num_views=8):
    ds = MultiViewRelightDataset(manifest_path, image_size=image_size, num_views=num_views)
    dl = DataLoader(ds, batch_size=1, shuffle=False)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = RelightingModel().to(device)
    model.load_state_dict(torch.load(model_ckpt, map_location=device))
    model.eval()

    os.makedirs(out_dir, exist_ok=True)

    with torch.no_grad():
        for idx, batch in enumerate(tqdm(dl, desc="relighting-test")):
            imgs = batch['images'].to(device)  # [1, V, 3, H, W]
            ref = imgs[:, 0]
            out = model(ref)

            # ذخیره ورودی و خروجی برای مقایسه
            scene_dir = os.path.join(out_dir, f"scene_{idx}")
            os.makedirs(scene_dir, exist_ok=True)

            ref_img = ref[0].cpu().permute(1,2,0).numpy()
            ref_img = (ref_img*255).astype("uint8")
            Image.fromarray(ref_img).save(os.path.join(scene_dir, "input.png"))

            out_img = out[0].cpu().permute(1,2,0).numpy()
            out_img = (out_img*255).astype("uint8")
            Image.fromarray(out_img).save(os.path.join(scene_dir, "relit.png"))

    print(f"Relit test images saved to {out_dir}")


# NeRF

import torch
import numpy as np
import os
from tqdm import tqdm
from PIL import Image


def test_nerf(model_ckpt, out_dir, res=64, num_samples=32):
    """
    تست مدل NeRF → رندر چند تصویر ساده
    """
    os.makedirs(out_dir, exist_ok=True)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = NeRF().to(device)   # ← همون معماری مقاله
    model.load_state_dict(torch.load(model_ckpt, map_location=device))
    model.eval()

    test_angles = [0, 90, 180, 270]

    for angle in tqdm(test_angles, desc="NeRF test rendering"):
        rays_o = torch.zeros(res*res, 3, device=device)
        rays_d = torch.randn(res*res, 3, device=device)

        t_vals = torch.linspace(0., 1., steps=num_samples, device=device)
        pts = rays_o.unsqueeze(1) + rays_d.unsqueeze(1) * t_vals.view(1, -1, 1)

        pts_flat = pts.reshape(-1, 3)
        dirs_flat = rays_d.unsqueeze(1).expand(-1, num_samples, -1).reshape(-1, 3)

        with torch.no_grad():
            rgb, sigma = model(pts_flat, dirs_flat)   # ✅ اینجا هر دو خروجی رو بگیر
            rgb = rgb.view(rays_o.shape[0], num_samples, 3)

            rgb_final = rgb.mean(dim=1).clamp(0,1)

        img = (rgb_final.view(res, res, 3).cpu().numpy() * 255).astype(np.uint8)
        Image.fromarray(img).save(os.path.join(out_dir, f"render_angle{angle}.png"))

    print(f"تست NeRF تمام شد → تصاویر در {out_dir} ذخیره شدند")



# MSE و PSNR
def psnr(mse_loss):
    return 10 * math.log10(1.0 / mse_loss)

def evaluate_relighting(manifest_path, model_ckpt, image_size=128, num_samples=10):
    ds = MultiViewRelightDataset(manifest_path, image_size=image_size)
    dl = DataLoader(ds, batch_size=1, shuffle=True)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = RelightingModel().to(device)
    model.load_state_dict(torch.load(model_ckpt))
    model.eval()

    loss_fn = nn.MSELoss()
    total_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(dl):
            if i >= num_samples: break
            imgs = batch['images'].to(device)
            ref = imgs[:,0]
            out = model(ref)
            loss = loss_fn(out, ref)
            total_loss += loss.item()

    avg_loss = total_loss / num_samples
    print(f"📊 Relighting Eval → MSE: {avg_loss:.6f}, PSNR: {psnr(avg_loss):.2f} dB")


In [ ]:
test_relighting(MANIFEST, os.path.join(RELIGHT_OUT, "relighting_model.pth"), "/content/test_relighting")
evaluate_relighting(MANIFEST, os.path.join(RELIGHT_OUT, "relighting_model.pth"))

test_nerf(os.path.join(NERF_OUT, "nerf_model.pth"), "/content/test_nerf")


relighting-test: 100%|██████████| 15/15 [00:00<00:00, 73.79it/s]


Relit test images saved to /content/test_relighting
📊 Relighting Eval → MSE: 0.050889, PSNR: 12.93 dB


NeRF test rendering: 100%|██████████| 4/4 [00:00<00:00, 13.49it/s]

تست NeRF تمام شد → تصاویر در /content/test_nerf ذخیره شدند


### Advanced

In [ ]:
!nvidia-smi
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!pip install diffusers==0.21.4 accelerate==0.25.0
!pip install nerfacc trimesh imageio[ffmpeg] matplotlib objaverse==0.1.7
!pip install --upgrade "transformers>=4.41.0,<5.0.0"

import objaverse.xl as oxl
import objaverse
import random, os

# پوشه‌ی ذخیره‌سازی فایل‌ها
save_dir = "objaverse_data"
os.makedirs(save_dir, exist_ok=True)

# تعداد پردازش‌های موازی (اختیاری)
num_procs = 4

# 1. بارگذاری داده‌های متادیتا (Annotations) به شکل DataFrame
annotations = oxl.get_annotations(download_dir=save_dir)

# 2. نمونه‌گیری تصادفی چند آبجکت
sample = annotations.sample(15).reset_index(drop=True)

# 3. دانلود از طریق XL API به همراه مسیر ذخیره‌سازی
objects = oxl.download_objects(
    objects=sample,
    download_dir=save_dir,
    processes=num_procs
)

print("Downloaded", len(objects), "assets.")

import numpy as np
from PIL import Image, ImageDraw

out_root = "dataset_objaverse"
os.makedirs(out_root, exist_ok=True)

uids = list(objaverse.load_uids())
sample_uids = random.sample(uids, 15)  # take 15 assets

for idx, uid in enumerate(sample_uids):
    asset_dir = os.path.join(out_root, f"asset_{idx}")
    os.makedirs(asset_dir, exist_ok=True)
    for view in range(8):
        img = Image.new("RGB", (128,128), (random.randint(0,255), random.randint(0,255), random.randint(0,255)))
        d = ImageDraw.Draw(img)
        d.text((10,60), f"{uid[:6]} v{view}", fill=(255,255,255))
        img.save(os.path.join(asset_dir, f"view_{view}.png"))
print("Generated dummy 8 views for each of 15 assets")

/bin/bash: line 1: nvidia-smi: command not found
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.35.1
    Uninstalling diffusers-0.35.1:
      Successfully uninstalled diffusers-0.35.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.10.1
    Uninstalling accelerate-1.10.1:
      Successfully uninstalled accelerate-1.10.1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.8 MB/s eta 0:00:00
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=049c01664

2025-09-02 20:35:41.500 | INFO     | objaverse.xl.github:_get_annotations:65 - Downloading https://huggingface.co/datasets/allenai/objaverse-xl/resolve/main/github/github.parquet to objaverse_data/github/github.parquet
2025-09-02 20:36:12.414 | INFO     | objaverse.xl.thingiverse:get_annotations:46 - Downloading https://huggingface.co/datasets/allenai/objaverse-xl/resolve/main/thingiverse/thingiverse.parquet to objaverse_data/thingiverse/thingiverse.parquet
2025-09-02 20:36:27.913 | INFO     | objaverse.xl.smithsonian:get_annotations:47 - Downloading https://huggingface.co/datasets/allenai/objaverse-xl/resolve/main/smithsonian/smithsonian.parquet to objaverse_data/smithsonian/smithsonian.parquet
2025-09-02 20:36:28.177 | INFO     | objaverse.xl.sketchfab:_get_annotations:52 - Downloading https://huggingface.co/datasets/allenai/objaverse-xl/resolve/main/sketchfab/sketchfab.parquet to objaverse_data/sketchfab/sketchfab.parquet
2025-09-02 20:36:34.658 | INFO     | objaverse.xl.thingiverse

Downloaded 1 assets.
Generated dummy 8 views for each of 15 assets


In [ ]:
!pip install -q torch torchvision tqdm einops

!pip install -q objaverse objaverse-xl

In [ ]:
import os, random, math, json, glob
from PIL import Image
import numpy as np
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from tqdm import tqdm
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cpu


In [ ]:
DATA_ROOT = "/content/dataset_objaverse"   # <-- مسیر دیتاست (هر asset یک فولدر شامل images/)
MANIFEST = "/content/relighting_manifest.json"
RELIGHT_OUT = "/content/relighting_out"
NERF_OUT = "/content/nerf_out"


RELIGHT_EPOCHS = 20
RELIGHT_BS = 2
NERF_EPOCHS = 50
NERF_BATCH_RAYS = 2048
NERF_N_SAMPLES = 64
IMAGE_SIZE = 128
NUM_VIEWS = 8
random.seed(42)

In [ ]:
os.makedirs(DATA_ROOT, exist_ok=True)

if len(os.listdir(DATA_ROOT)) == 0:
    print("DATA_ROOT empty — generating dummy assets...")
    import random
    for a in range(8):
        ad = os.path.join(DATA_ROOT, f"asset_{a}")
        os.makedirs(os.path.join(ad, "images"), exist_ok=True)
        for v in range(NUM_VIEWS):
            img = Image.new("RGB", (IMAGE_SIZE, IMAGE_SIZE),
                            (random.randint(0,255), random.randint(0,255), random.randint(0,255)))
            d = Image.Draw.Draw(img)
            d.text((8, IMAGE_SIZE//2), f"asset{a}_v{v}", fill=(255,255,255))
            img.save(os.path.join(ad, "images", f"view_{v}.png"))
    print("Dummy assets generated.")
else:
    print("DATA_ROOT non-empty; skipping dummy generation.")

DATA_ROOT non-empty; skipping dummy generation.


In [ ]:
def make_spherical_transforms(n_views=NUM_VIEWS, radius=1.0):
    frames = []
    for i in range(n_views):
        theta = 2*math.pi * i / n_views
        cam_pos = [radius * math.sin(theta), 0.0, radius * math.cos(theta)]
        T = np.eye(4)
        T[:3,3] = cam_pos
        frames.append({
            "file_path": f"images/view_{i}.png",
            "transform_matrix": T.tolist()
        })
    return {"camera_angle_x": 0.69, "frames": frames}

for asset in sorted(os.listdir(DATA_ROOT)):
    ad = os.path.join(DATA_ROOT, asset)
    if not os.path.isdir(ad): continue
    tfp = os.path.join(ad, "transforms.json")
    if os.path.exists(tfp):
        continue
    else:
        d = make_spherical_transforms(NUM_VIEWS)
        with open(tfp, "w") as f:
            json.dump(d, f, indent=2)
print("Transforms created or verified for each asset.")

Transforms created or verified for each asset.


In [ ]:
DATA_ROOT = "/content/dataset_objaverse"   # مسیر دیتاست ساختگی
MANIFEST = "/content/relighting_manifest.json"

scenes = []
for asset in sorted(os.listdir(DATA_ROOT)):
    ad = os.path.join(DATA_ROOT, asset)
    if not os.path.isdir(ad):
        continue

    images = sorted([p for p in os.listdir(ad) if p.endswith(".png") or p.endswith(".jpg")])
    if len(images) == 0:
        continue
    scenes.append({
        "folder": ad,
        "images": images,
        "reference_index": 0
    })

with open(MANIFEST, "w") as f:
    json.dump({"scenes": scenes}, f, indent=2)

print(f"Manifest saved to {MANIFEST}, with {len(scenes)} scenes")


✅ Manifest saved to /content/relighting_manifest.json, with 15 scenes


In [ ]:
class MultiViewRelightDataset(Dataset):
    def __init__(self, manifest_path, image_size=IMAGE_SIZE, num_views=NUM_VIEWS, transform=None):
        with open(manifest_path, 'r') as f:
            manifest = json.load(f)
        self.scenes = manifest['scenes']
        self.image_size = image_size
        self.num_views = num_views
        self.transform = transform
    def __len__(self):
        return len(self.scenes)
    def __getitem__(self, idx):
        scene = self.scenes[idx]
        folder = scene['folder']
        images = scene['images']
        ref = scene.get('reference_index', 0)
        others = [i for i in range(len(images)) if i != ref]
        pick_rest = self.num_views - 1
        picked = [ref] + random.sample(others, min(pick_rest, len(others)))
        arrs = []
        for i in picked:
            img = Image.open(os.path.join(folder, images[i])).convert('RGB').resize((self.image_size,self.image_size))
            if self.transform:
                arrs.append(self.transform(img))
            else:
                arrs.append(transforms.ToTensor()(img))
        arrs = torch.stack(arrs, dim=0)  # [num_views, C, H, W]
        shading_embed = torch.zeros(1)
        return {"images": arrs, "refer_idx": 0, "asset_folder": folder, "shading": shading_embed}

In [ ]:
# ResBlock
class ResBlock(nn.Module):
    def __init__(self, in_ch, out_ch, downsample=False):
        super().__init__()
        self.downsample = downsample
        mid_ch = out_ch
        self.conv1 = nn.Conv2d(in_ch, mid_ch, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(mid_ch)
        self.conv2 = nn.Conv2d(mid_ch, out_ch, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_ch)
        if in_ch != out_ch or downsample:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=1, bias=False),
                nn.BatchNorm2d(out_ch)
            )
        else:
            self.shortcut = nn.Identity()
        self.act = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d(2) if downsample else nn.Identity()

    def forward(self, x):
        out = self.act(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        sc = self.shortcut(x)
        out = self.act(out + sc)
        out = self.pool(out)
        return out

# Attention Fusion
class ViewFusion(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.project = nn.Conv2d(in_ch, out_ch, kernel_size=1)
        self.att_conv = nn.Conv2d(out_ch, 1, kernel_size=1)

    def forward(self, view_feats):
        stacked = torch.stack(view_feats, dim=1)  # [B, V, C, H, W]
        B,V,C,H,W = stacked.shape
        merged = stacked.view(B*V, C, H, W)
        proj = self.project(merged)  # [B*V, out_ch, H, W]
        proj = proj.view(B, V, -1, H, W)  # [B,V,out_ch,H,W]
        att_logits = self.att_conv(proj.view(B*V, -1, H, W)).view(B, V, H, W)  # [B,V,H,W]
        att = torch.softmax(att_logits.mean(dim=[2,3]), dim=1)  # [B,V]
        att = att.view(B, V, 1, 1, 1)
        fused = (proj * att).sum(dim=1)  # [B, out_ch, H, W]
        return fused

# ResUNetRelight with multi-view fusion
class ResUNetRelight(nn.Module):
    def __init__(self, in_ch=3, base=64, num_views=NUM_VIEWS):
        super().__init__()
        self.num_views = num_views
        self.view_shallow = nn.Sequential(
            nn.Conv2d(in_ch, base, 3, padding=1), nn.ReLU(),
            nn.Conv2d(base, base, 3, padding=1), nn.ReLU()
        )
        # fusion
        self.fusion = ViewFusion(base, base)
        # encoder residual blocks
        self.enc1 = ResBlock(base, base, downsample=False)     # out: base
        self.enc2 = ResBlock(base, base*2, downsample=True)    # out: base*2
        self.enc3 = ResBlock(base*2, base*4, downsample=True)  # out: base*4
        self.enc4 = ResBlock(base*4, base*8, downsample=True)  # out: base*8
        # bottleneck
        self.bottleneck = nn.Sequential(
            nn.Conv2d(base*8, base*8, 3, padding=1), nn.ReLU(),
            nn.Conv2d(base*8, base*8, 3, padding=1), nn.ReLU()
        )
        # decoder (upsample + skip)
        self.up4 = nn.ConvTranspose2d(base*8, base*4, 2, stride=2)
        self.dec4 = nn.Sequential(nn.Conv2d(base*8, base*4, 3, padding=1), nn.ReLU())
        self.up3 = nn.ConvTranspose2d(base*4, base*2, 2, stride=2)
        self.dec3 = nn.Sequential(nn.Conv2d(base*4, base*2, 3, padding=1), nn.ReLU())
        self.up2 = nn.ConvTranspose2d(base*2, base, 2, stride=2)
        self.dec2 = nn.Sequential(nn.Conv2d(base*2, base, 3, padding=1), nn.ReLU())
        self.outc = nn.Conv2d(base, 3, 1)

    def forward(self, views):  # views: [B, V, C, H, W]
        B,V,C,H,W = views.shape
        view_feats = []
        for v in range(V):
            x = views[:,v]  # [B,C,H,W]
            x = self.view_shallow(x)
            view_feats.append(x)
        fused = self.fusion(view_feats)  # [B, base, H, W]
        e1 = self.enc1(fused)
        e2 = self.enc2(e1)
        e3 = self.enc3(e2)
        e4 = self.enc4(e3)
        b = self.bottleneck(e4)
        u4 = self.up4(b)
        d4 = self.dec4(torch.cat([u4, e4], dim=1))
        u3 = self.up3(d4)
        d3 = self.dec3(torch.cat([u3, e3], dim=1))
        u2 = self.up2(d3)
        d2 = self.dec2(torch.cat([u2, e2], dim=1))
        out = torch.sigmoid(self.outc(d2))
        return out


In [ ]:
class RelightingModel(nn.Module):
    def __init__(self):
        super(RelightingModel, self).__init__()
        self.enc1 = nn.Conv2d(3, 32, 3, stride=2, padding=1)
        self.enc2 = nn.Conv2d(32, 64, 3, stride=2, padding=1)
        self.enc3 = nn.Conv2d(64, 128, 3, stride=2, padding=1)

        self.fc1 = nn.Linear(128*16*16, 512)
        self.fc2 = nn.Linear(512, 128*16*16)

        self.dec1 = nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1)
        self.dec2 = nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1)
        self.dec3 = nn.ConvTranspose2d(32, 3, 4, stride=2, padding=1)

    def forward(self, x):
        h = F.relu(self.enc1(x))   # [B,32,H/2,W/2]
        h = F.relu(self.enc2(h))   # [B,64,H/4,W/4]
        h = F.relu(self.enc3(h))   # [B,128,H/8,W/8]

        B, C, H, W = h.shape
        h = h.view(B, -1)
        h = F.relu(self.fc1(h))
        h = F.relu(self.fc2(h))
        h = h.view(B, 128, H, W)

        h = F.relu(self.dec1(h))
        h = F.relu(self.dec2(h))
        out = torch.sigmoid(self.dec3(h))
        return out


In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from PIL import Image
import numpy as np

def train_relighting(manifest_path, out_dir, epochs=3, bs=1, image_size=128, num_views=8, save_interval=1):
    ds = MultiViewRelightDataset(manifest_path, image_size=image_size, num_views=num_views)
    dl = DataLoader(ds, batch_size=bs, shuffle=True)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = RelightingModel().to(device)
    opt = torch.optim.Adam(model.parameters(), 1e-3)
    loss_fn = nn.MSELoss()

    os.makedirs(out_dir, exist_ok=True)

    for epoch in range(epochs):
        total_loss = 0
        pbar = tqdm(dl, desc=f"Epoch {epoch+1}/{epochs}")
        for batch in pbar:
            imgs = batch['images'].to(device)  # [B,V,3,H,W]
            ref = model(imgs)
            out = model(ref)
            loss = loss_fn(out, ref)

            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(dl)
        print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f}")

        save_epoch_dir = os.path.join(out_dir, f"epoch_{epoch+1}")
        os.makedirs(save_epoch_dir, exist_ok=True)
        ref_img = ref[0].detach().cpu().permute(1,2,0).numpy()
        out_img = out[0].detach().cpu().permute(1,2,0).numpy()
        Image.fromarray((ref_img*255).astype("uint8")).save(os.path.join(save_epoch_dir, "input.png"))
        Image.fromarray((out_img*255).astype("uint8")).save(os.path.join(save_epoch_dir, "relit.png"))

    torch.save(model.state_dict(), os.path.join(out_dir, 'relighting_model.pth'))
    print(f"Training finished. Model saved to {out_dir}")


In [ ]:
class NeRFDataset(Dataset):
    def __init__(self, relit_root, img_size=IMAGE_SIZE, n_rays=None):
        self.items = []  # list of dicts: {image_path, pose, asset}
        self.img_size = img_size
        for asset in sorted(os.listdir(relit_root)):
            ad = os.path.join(relit_root, asset)
            if not os.path.isdir(ad): continue
            # images
            imgs = sorted([f for f in os.listdir(ad) if f.endswith(".png") or f.endswith(".jpg")])
            if len(imgs) == 0: continue
            # try to find transforms.json in original DATA_ROOT
            original_asset = os.path.join(DATA_ROOT, asset)  # our naming convention
            tfp = os.path.join(original_asset, "transforms.json")
            poses = {}
            if os.path.exists(tfp):
                data = json.load(open(tfp))
                for fr in data["frames"]:
                    poses[os.path.basename(fr["file_path"])] = fr["transform_matrix"]
            for imf in imgs:
                path = os.path.join(ad, imf)
                pose = poses.get(imf, None)
                self.items.append({"path": path, "pose": pose, "asset": asset})
    def __len__(self):
        return len(self.items)
    def __getitem__(self, idx):
        it = self.items[idx]
        img = Image.open(it["path"]).convert("RGB").resize((self.img_size,self.img_size))
        img = transforms.ToTensor()(img)
        pose = it["pose"]  # may be None
        return {"image": img, "pose": pose, "path": it["path"]}

In [ ]:
# positional encoding helper
class PosEncoding(nn.Module):
    def __init__(self, num_freqs):
        super().__init__()
        self.num_freqs = num_freqs
        self.freq_bands = 2.0 ** torch.arange(0, num_freqs).float()

    def forward(self, x):
        # x: [..., D]
        out = [x]
        for freq in self.freq_bands:
            out.append(torch.sin(x * freq * math.pi))
            out.append(torch.cos(x * freq * math.pi))
        return torch.cat(out, dim=-1)

# NeRF MLP block (shared)
def mlp_block(in_ch, out_ch):
    return nn.Sequential(nn.Linear(in_ch, out_ch), nn.ReLU(inplace=True))

# Full NeRF MLP (coarse or fine)
class NeRF_MLP(nn.Module):
    def __init__(self, D=8, W=256, input_ch=63, input_ch_dir=27, skips=[4], use_viewdirs=True):
        super().__init__()
        self.D = D; self.W = W; self.skips = skips; self.use_viewdirs = use_viewdirs
        self.pts_linears = nn.ModuleList()
        for i in range(D):
            if i == 0:
                self.pts_linears.append(nn.Linear(input_ch, W))
            elif i in skips:
                self.pts_linears.append(nn.Linear(W + input_ch, W))
            else:
                self.pts_linears.append(nn.Linear(W, W))
        # feature -> sigma + feature
        self.sigma_layer = nn.Linear(W, 1)
        self.feature_layer = nn.Linear(W, W)
        # view-dependent rgb head
        if use_viewdirs:
            self.rgb_layer = nn.Sequential(nn.Linear(W + input_ch_dir, W//2), nn.ReLU(), nn.Linear(W//2, 3))
        else:
            self.rgb_layer = nn.Linear(W, 3)

    def forward(self, x, viewdirs=None):
        h = x
        for i, l in enumerate(self.pts_linears):
            if i in self.skips and i != 0:
                h = torch.cat([h, x], dim=-1)
            h = F.relu(l(h))
        sigma = F.relu(self.sigma_layer(h))
        feat = self.feature_layer(h)
        if self.use_viewdirs and viewdirs is not None:
            h2 = torch.cat([feat, viewdirs], dim=-1)
            rgb = torch.sigmoid(self.rgb_layer(h2))
        else:
            rgb = torch.sigmoid(self.rgb_layer(feat))
        return rgb, sigma

# sampling helpers (stratified + importance)
def sample_stratified(n_rays, n_samples, near, far, device):
    t_vals = torch.linspace(0.0, 1.0, n_samples, device=device)
    mids = 0.5 * (t_vals[:-1] + t_vals[1:])
    upper = torch.cat([mids, t_vals[-1].unsqueeze(0)], 0)
    lower = torch.cat([t_vals[0].unsqueeze(0), mids], 0)
    t_rand = torch.rand((n_rays, n_samples), device=device)
    t_vals = lower.unsqueeze(0) + (upper - lower).unsqueeze(0) * t_rand
    return t_vals  # in [0,1] param domain

def sample_pdf(bins, weights, n_samples, device):
    # bins: [N_rays, N_coarse] (edges), weights: [N_rays, N_coarse]
    weights = weights + 1e-5
    pdf = weights / torch.sum(weights, dim=-1, keepdim=True)
    cdf = torch.cumsum(pdf, dim=-1)
    cdf = torch.cat([torch.zeros_like(cdf[..., :1]), cdf], dim=-1)
    u = torch.rand((cdf.shape[0], n_samples), device=device)
    # inverse cdf (linear search)
    inds = torch.searchsorted(cdf, u, right=True) - 1
    inds = inds.clamp(0, cdf.shape[-1]-2)
    cdf_g0 = torch.gather(cdf, 1, inds)
    cdf_g1 = torch.gather(cdf, 1, inds+1)
    bins_g0 = torch.gather(bins, 1, inds)
    bins_g1 = torch.gather(bins, 1, inds+1)
    t = (u - cdf_g0) / (cdf_g1 - cdf_g0 + 1e-8)
    samples = bins_g0 + t * (bins_g1 - bins_g0)
    return samples

# combined renderer using coarse+fine
class NeRFCoarseFine:
    def __init__(self, device, N_coarse=64, N_fine=64, near=0.1, far=4.0, use_viewdirs=True):
        self.device = device
        self.Nc = N_coarse
        self.Nf = N_fine
        self.near = near; self.far = far
        self.use_viewdirs = use_viewdirs
        # encoders
        self.pe_pos = PosEncoding(num_freqs=10)
        self.pe_dir = PosEncoding(num_freqs=4)
        # models
        input_ch = 3 + 2*10*3
        input_ch_dir = 3 + 2*4*3
        self.coarse = NeRF_MLP(D=8, W=256, input_ch=input_ch, input_ch_dir=input_ch_dir, use_viewdirs=use_viewdirs).to(device)
        self.fine = NeRF_MLP(D=8, W=256, input_ch=input_ch, input_ch_dir=input_ch_dir, use_viewdirs=use_viewdirs).to(device)

    def render_rays(self, origins, dirs):
        N = origins.shape[0]
        device = self.device
        # coarse stratified sampling in [near, far]
        t_coarse = sample_stratified(N, self.Nc, self.near, self.far, device)  # [N, Nc], in [0,1]
        z_coarse = self.near + (self.far - self.near) * t_coarse  # [N, Nc]
        pts = origins.unsqueeze(1) + dirs.unsqueeze(1) * z_coarse.unsqueeze(-1)  # [N,Nc,3]
        pts_flat = pts.reshape(-1,3)
        # encode pos and dirs
        pe_pts = self.pe_pos(torch.tensor(pts_flat, device=device))
        viewdirs = dirs / torch.norm(dirs, dim=-1, keepdim=True)
        viewdirs_enc = self.pe_dir(viewdirs).unsqueeze(1).expand(-1, self.Nc, -1).reshape(-1, viewdirs.shape[-1]*(2*4+1))
        # forward coarse
        rgb_c, sigma_c = self.coarse(pe_pts, viewdirs_enc.to(device))
        rgb_c = rgb_c.view(N, self.Nc, 3)
        sigma_c = sigma_c.view(N, self.Nc, 1)
        # rendering coarse
        comp_c, weights_c = volume_rendering(rgb_c, sigma_c, z_coarse, dirs)
        bins = 0.5*(z_coarse[:, :-1] + z_coarse[:, 1:])  # [N, Nc-1]
        weights_for_pdf = weights_c[:, :-1].detach()  # [N, Nc-1]
        z_fine_norm = sample_pdf(bins, weights_for_pdf, self.Nf, device)  # [N, Nf] in [near, far]
        z_all = torch.cat([z_coarse, z_fine_norm], dim=1)
        z_all, _ = torch.sort(z_all, dim=1)
        pts_all = origins.unsqueeze(1) + dirs.unsqueeze(1) * z_all.unsqueeze(-1)
        pts_all_flat = pts_all.reshape(-1,3)
        pe_pts_all = self.pe_pos(torch.tensor(pts_all_flat, device=device))
        viewdirs_enc_all = self.pe_dir(viewdirs).unsqueeze(1).expand(-1, z_all.shape[1], -1).reshape(-1, viewdirs.shape[-1]*(2*4+1))
        rgb_f, sigma_f = self.fine(pe_pts_all, viewdirs_enc_all.to(device))
        rgb_f = rgb_f.view(N, z_all.shape[1], 3)
        sigma_f = sigma_f.view(N, z_all.shape[1], 1)
        comp_f, weights_f = volume_rendering(rgb_f, sigma_f, z_all, dirs)
        return comp_f, weights_c, z_coarse, rgb_c


In [ ]:
def get_ray_directions(H, W, focal):
    i, j = torch.meshgrid(torch.arange(W), torch.arange(H), indexing='xy')
    i = i.t().float(); j = j.t().float()
    dirs = torch.stack([(i - W*0.5)/focal, -(j - H*0.5)/focal, -torch.ones_like(i)], -1)  # [H,W,3]
    return dirs  # camera coords

def sample_points_along_rays(origins, directions, near, far, N_samples):
    # origins, directions: [num_rays, 3]
    t_vals = torch.linspace(near, far, N_samples).to(origins.device)
    t_vals = t_vals.expand(origins.shape[0], N_samples)
    pts = origins.unsqueeze(1) + directions.unsqueeze(1) * t_vals.unsqueeze(-1)  # [num_rays, N_samples, 3]
    return pts, t_vals

def volume_rendering(rgb, sigma, z_vals, dirs):
    # rgb: [num_rays, N_samples, 3], sigma: [num_rays, N_samples, 1]
    deltas = z_vals[:,1:] - z_vals[:,:-1]  # [num_rays, N-1]
    delta_last = 1e10 * torch.ones_like(deltas[:,:1])
    deltas = torch.cat([deltas, delta_last], dim=1)  # [num_rays, N]
    alpha = 1.0 - torch.exp(-sigma.squeeze(-1) * deltas)
    weights = alpha * torch.cumprod(torch.cat([torch.ones((alpha.shape[0],1), device=alpha.device), 1.0 - alpha + 1e-10], dim=1), dim=1)[:,:-1]
    # weights: [num_rays, N_samples]
    comp_rgb = (weights.unsqueeze(-1) * rgb).sum(dim=1)  # [num_rays,3]
    return comp_rgb, weights


In [ ]:
def load_transforms_for_asset(asset_folder):
    tfp = os.path.join(asset_folder, "transforms.json")
    if not os.path.exists(tfp):
        return None
    return json.load(open(tfp))

def train_nerf_on_relit_enhanced(relit_root, out_dir, steps=2000, batch_rays=1024, N_coarse=64, N_fine=64, H=IMAGE_SIZE, W=IMAGE_SIZE):
    ds = NeRFDataset(relit_root, img_size=H)
    if len(ds) == 0:
        print("No relit data found in", relit_root); return
    device_local = device
    renderer = NeRFCoarseFine(device_local, N_coarse=N_coarse, N_fine=N_fine)
    optimizer = torch.optim.Adam(list(renderer.coarse.parameters()) + list(renderer.fine.parameters()), lr=5e-4)
    loss_fn = nn.MSELoss()
    os.makedirs(out_dir, exist_ok=True)

    ray_data = []
    for it in ds.items:
        img_t = transforms.ToTensor()(Image.open(it["path"]).convert("RGB").resize((W,H)))
        pose = it["pose"]
        if pose is None:
            pose_mat = np.eye(4).tolist()
        else:
            pose_mat = pose
        camera_angle_x = 0.69
        focal = 0.5 * W / math.tan(0.5 * camera_angle_x)
        dirs_cam = get_ray_directions(H,W,focal).reshape(-1,3).to(device_local)
        pose_mat = torch.tensor(pose_mat, dtype=torch.float32).to(device_local)
        R = pose_mat[:3,:3]; t = pose_mat[:3,3]
        dirs_world = (dirs_cam @ R.t())
        origins = t.unsqueeze(0).expand(dirs_world.shape[0],3)
        colors = img_t.permute(1,2,0).reshape(-1,3).to(device_local)
        ray_data.append({"origins": origins, "dirs": dirs_world, "colors": colors})
    print("Prepared ray data from", len(ray_data), "images.")

    for step in range(steps):
        idx_img = random.randrange(len(ray_data))
        rd = ray_data[idx_img]
        num_rays_total = rd["dirs"].shape[0]
        sel_idx = torch.randperm(num_rays_total)[:batch_rays]
        origins = rd["origins"][sel_idx].to(device_local)
        dirs = rd["dirs"][sel_idx].to(device_local)
        target_rgb = rd["colors"][sel_idx].to(device_local)
        comp_rgb, w_c, z_c, rgb_c = renderer.render_rays(origins, dirs)
        loss = loss_fn(comp_rgb, target_rgb)
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        if step % 50 == 0:
            print(f"[NeRF] step {step}/{steps} loss {loss.item():.6f}")
    # save models
    torch.save(renderer.coarse.state_dict(), os.path.join(out_dir, "nerf_coarse.pth"))
    torch.save(renderer.fine.state_dict(), os.path.join(out_dir, "nerf_fine.pth"))
    print("Saved NeRF coarse & fine.")
    return out_dir


In [ ]:
# 1) Relighting
os.makedirs(RELIGHT_OUT, exist_ok=True)
train_relighting(MANIFEST, RELIGHT_OUT, epochs=RELIGHT_EPOCHS, bs=RELIGHT_BS, image_size=IMAGE_SIZE, num_views=NUM_VIEWS)

for asset in sorted(os.listdir(RELIGHT_OUT))[:5]:
    print("Relit outputs for:", asset, "->", os.listdir(os.path.join(RELIGHT_OUT, asset))[:5])

# 2) Train NeRF
os.makedirs(NERF_OUT, exist_ok=True)
train_nerf_on_relit(RELIGHT_OUT, NERF_OUT, steps=NERF_EPOCHS, batch_rays=NERF_BATCH_RAYS, N_samples=NERF_N_SAMPLES, H=IMAGE_SIZE, W=IMAGE_SIZE)

print("Pipeline finished.")


In [ ]:
# Relighting

def test_relighting(manifest_path, model_ckpt, out_dir, image_size=128, num_views=8):
    ds = MultiViewRelightDataset(manifest_path, image_size=image_size, num_views=num_views)
    dl = DataLoader(ds, batch_size=1, shuffle=False)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = RelightingModel().to(device)
    model.load_state_dict(torch.load(model_ckpt, map_location=device))
    model.eval()

    os.makedirs(out_dir, exist_ok=True)

    with torch.no_grad():
        for idx, batch in enumerate(tqdm(dl, desc="relighting-test")):
            imgs = batch['images'].to(device)  # [1, V, 3, H, W]
            ref = imgs[:, 0]
            out = model(ref)

            scene_dir = os.path.join(out_dir, f"scene_{idx}")
            os.makedirs(scene_dir, exist_ok=True)

            ref_img = ref[0].cpu().permute(1,2,0).numpy()
            ref_img = (ref_img*255).astype("uint8")
            Image.fromarray(ref_img).save(os.path.join(scene_dir, "input.png"))

            out_img = out[0].cpu().permute(1,2,0).numpy()
            out_img = (out_img*255).astype("uint8")
            Image.fromarray(out_img).save(os.path.join(scene_dir, "relit.png"))

    print(f"Relit test images saved to {out_dir}")


# NeRF => Test

import torch
import torch.nn.functional as F
import numpy as np
import os
from PIL import Image
from tqdm import tqdm

def test_nerf(model_ckpt, out_dir, res=64, num_samples=64):
    os.makedirs(out_dir, exist_ok=True)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    model = NeRF().to(device)
    model.load_state_dict(torch.load(model_ckpt))
    model.eval()


    H, W = res, res
    rays_o = torch.zeros(H*W, 3).to(device)  # مبدأ پرتوها (0,0,0)
    dirs = []
    for y in range(H):
        for x in range(W):
            dirs.append([ (x - W/2)/W, -(y - H/2)/H, -1.0 ])  # ساده: رو به -z
    rays_d = torch.tensor(dirs, dtype=torch.float32).to(device)


    rendered = []
    for i in tqdm(range(0, H*W, 1024), desc="NeRF test rendering"):
        batch_rays_o = rays_o[i:i+1024]
        batch_rays_d = rays_d[i:i+1024]

        t_vals = torch.linspace(0., 1., steps=num_samples).to(device)
        pts = batch_rays_o[:, None, :] + batch_rays_d[:, None, :] * t_vals[None,:,None]  # [N_rays, N_samples, 3]
        pts_flat = pts.reshape(-1, 3)

        with torch.no_grad():
            rgb = model(pts_flat)  # [N_rays*N_samples, 3]
            rgb = rgb.view(batch_rays_o.shape[0], num_samples, 3)

            rgb_final = rgb.mean(dim=1)

        rendered.append(rgb_final.cpu())

    rendered = torch.cat(rendered, dim=0).numpy().reshape(H, W, 3)
    rendered = (rendered * 255).clip(0,255).astype(np.uint8)

    out_path = os.path.join(out_dir, "nerf_test.png")
    Image.fromarray(rendered).save(out_path)
    print(f"NeRF test image saved at {out_path}")


# MSE و PSNR

def psnr(mse_loss):
    return 10 * math.log10(1.0 / mse_loss)

def evaluate_relighting(manifest_path, model_ckpt, image_size=128, num_samples=10):
    ds = MultiViewRelightDataset(manifest_path, image_size=image_size)
    dl = DataLoader(ds, batch_size=1, shuffle=True)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = RelightingModel().to(device)
    model.load_state_dict(torch.load(model_ckpt))
    model.eval()

    loss_fn = nn.MSELoss()
    total_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(dl):
            if i >= num_samples: break
            imgs = batch['images'].to(device)
            ref = imgs[:,0]
            out = model(ref)
            loss = loss_fn(out, ref)
            total_loss += loss.item()

    avg_loss = total_loss / num_samples
    print(f"Relighting Eval → MSE: {avg_loss:.6f}, PSNR: {psnr(avg_loss):.2f} dB")


In [ ]:
test_relighting(MANIFEST, os.path.join(RELIGHT_OUT, "relighting_model.pth"), "/content/test_relighting")
evaluate_relighting(MANIFEST, os.path.join(RELIGHT_OUT, "relighting_model.pth"))

test_nerf(os.path.join(NERF_OUT, "nerf_model.pth"), "/content/test_nerf")
